In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from statistics import mode
from sklearn.metrics import confusion_matrix, auc, f1_score, classification_report, roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC

In [2]:
df = pd.read_excel('COVID-19.xlsx')

In [11]:
pd.set_option("display.max_columns", None)

In [12]:
df.head()

,Sno,age,gender,body_temperature,Dry_Cough,sour_throat,weakness,breathing_problem,drowsiness,pain_in_chest,travel_history_to_infected_countries,diabetes,heart_disease,lung_disease,stroke_or_reduced_immunity,symptoms_progressed,high_blood_pressue,kidney_disease,change_in_appetide,Loss_of_sense_of_smell,Corona_result
0,1,-1.175726,1,-0.915662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,-1.247169,1,-0.659801,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.324801,0,1.259162,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1
3,4,0.253147,0,-0.020147,0,0,0,0,1,1,1,1,0,0,0,1,1,0,1,0,2
4,5,-0.246959,1,-0.531870,0,1,0,1,0,0,1,0,0,1,1,1,0,0,0,1,2


In [4]:
df.columns =[c.replace(' ', '_') for c in df.columns]

In [5]:
df.head()

,Sno,age,gender,body_temperature,Dry_Cough,sour_throat,weakness,breathing_problem,drowsiness,pain_in_chest,...,diabetes,heart_disease,lung_disease,stroke_or_reduced_immunity,symptoms_progressed,high_blood_pressue,kidney_disease,change_in_appetide,Loss_of_sense_of_smell,Corona_result
0,1,20,1,98.6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,19,1,99.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,55,0,102.0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,1
3,4,40,0,100.0,0,0,0,0,1,1,...,1,0,0,0,1,1,0,1,0,2
4,5,33,1,99.2,0,1,0,1,0,0,...,0,0,1,1,1,0,0,0,1,2


In [6]:
df.gender.groupby([df.Corona_result, df.gender]).count()

Corona_result  gender
0              0         18
               1         15
1              0         30
               1         39
2              0         12
               1         13
Name: gender, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 21 columns):
Sno                                     127 non-null int64
age                                     127 non-null int64
gender                                  127 non-null int64
body_temperature                        127 non-null float64
Dry_Cough                               127 non-null int64
sour_throat                             127 non-null int64
weakness                                127 non-null int64
breathing_problem                       127 non-null int64
drowsiness                              127 non-null int64
pain_in_chest                           127 non-null int64
travel_history_to_infected_countries    127 non-null int64
diabetes                                127 non-null int64
heart_disease                           127 non-null int64
lung_disease                            127 non-null int64
stroke_or_reduced_immunity              127 non-null int64
symptoms

In [9]:
sc = StandardScaler()
df[['age', 'body_temperature']] = sc.fit_transform(df[['age', 'body_temperature']])

In [13]:
df.head()

,Sno,age,gender,body_temperature,Dry_Cough,sour_throat,weakness,breathing_problem,drowsiness,pain_in_chest,travel_history_to_infected_countries,diabetes,heart_disease,lung_disease,stroke_or_reduced_immunity,symptoms_progressed,high_blood_pressue,kidney_disease,change_in_appetide,Loss_of_sense_of_smell,Corona_result
0,1,-1.175726,1,-0.915662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,-1.247169,1,-0.659801,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.324801,0,1.259162,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1
3,4,0.253147,0,-0.020147,0,0,0,0,1,1,1,1,0,0,0,1,1,0,1,0,2
4,5,-0.246959,1,-0.531870,0,1,0,1,0,0,1,0,0,1,1,1,0,0,0,1,2


In [14]:
df.shape

(127, 21)

In [33]:
x = df.iloc[:, 1:-1]

In [48]:
columns = [
 'gender',
 'Dry_Cough',
 'sour_throat',
 'weakness',
 'breathing_problem',
 'drowsiness',
 'pain_in_chest',
 'travel_history_to_infected_countries',
 'diabetes',
 'heart_disease',
 'lung_disease',
 'stroke_or_reduced_immunity',
 'symptoms_progressed',
 'high_blood_pressue',
 'kidney_disease',
 'change_in_appetide',
 'Loss_of_sense_of_smell']
x = pd.get_dummies(x, columns=columns)

In [49]:
x.columns

Index([u'age', u'body_temperature', u'gender_0', u'gender_1', u'Dry_Cough_0',
       u'Dry_Cough_1', u'sour_throat_0', u'sour_throat_1', u'weakness_0',
       u'weakness_1', u'breathing_problem_0', u'breathing_problem_1',
       u'drowsiness_0', u'drowsiness_1', u'pain_in_chest_0',
       u'pain_in_chest_1', u'travel_history_to_infected_countries_0',
       u'travel_history_to_infected_countries_1', u'diabetes_0', u'diabetes_1',
       u'heart_disease_0', u'heart_disease_1', u'lung_disease_0',
       u'lung_disease_1', u'stroke_or_reduced_immunity_0',
       u'stroke_or_reduced_immunity_1', u'symptoms_progressed_0',
       u'symptoms_progressed_1', u'high_blood_pressue_0',
       u'high_blood_pressue_1', u'kidney_disease_0', u'kidney_disease_1',
       u'change_in_appetide_0', u'change_in_appetide_1',
       u'Loss_of_sense_of_smell_0', u'Loss_of_sense_of_smell_1'],
      dtype='object')

In [34]:
y = df.iloc[:, -1]

In [35]:
x.shape

(127, 19)

In [50]:
x.head()

,age,body_temperature,gender_0,gender_1,Dry_Cough_0,Dry_Cough_1,sour_throat_0,sour_throat_1,weakness_0,weakness_1,breathing_problem_0,breathing_problem_1,drowsiness_0,drowsiness_1,pain_in_chest_0,pain_in_chest_1,travel_history_to_infected_countries_0,travel_history_to_infected_countries_1,diabetes_0,diabetes_1,heart_disease_0,heart_disease_1,lung_disease_0,lung_disease_1,stroke_or_reduced_immunity_0,stroke_or_reduced_immunity_1,symptoms_progressed_0,symptoms_progressed_1,high_blood_pressue_0,high_blood_pressue_1,kidney_disease_0,kidney_disease_1,change_in_appetide_0,change_in_appetide_1,Loss_of_sense_of_smell_0,Loss_of_sense_of_smell_1
0,-1.175726,-0.915662,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
1,-1.247169,-0.659801,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
2,1.324801,1.259162,1,0,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0
3,0.253147,-0.020147,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,1,0,0,1,1,0
4,-0.246959,-0.531870,0,1,1,0,0,1,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1


In [72]:
model = LogisticRegression()

In [38]:
x.isnull().sum()

age                                     0
gender                                  0
body_temperature                        0
Dry_Cough                               0
sour_throat                             0
weakness                                0
breathing_problem                       0
drowsiness                              0
pain_in_chest                           0
travel_history_to_infected_countries    0
diabetes                                0
heart_disease                           0
lung_disease                            0
stroke_or_reduced_immunity              0
symptoms_progressed                     0
high_blood_pressue                      0
kidney_disease                          0
change_in_appetide                      0
Loss_of_sense_of_smell                  0
dtype: int64

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.3)

In [93]:
model = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=0, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print confusion_matrix(y_test, y_pred)

[[ 4  8  0]
 [ 0 19  0]
 [ 0  6  2]]


In [94]:
model.score(x_test, y_test)

0.6410256410256411

In [54]:
x_test.shape

(39, 36)

In [68]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
y_pred = rf_model.predict(x_test)
print confusion_matrix(y_test, y_pred)

[[ 5  6  1]
 [ 1 15  3]
 [ 0  6  2]]


In [69]:
rf_model.score(x_test, y_test)

0.5641025641025641

In [57]:
rf_model.feature_importances_

array([0.06308755, 0.07199217, 0.03221172, 0.00706658, 0.02503036,
       0.01208646, 0.01464057, 0.04580961, 0.01922919, 0.03293751,
       0.04928964, 0.02568488, 0.07076733, 0.0185253 , 0.00922137,
       0.02692123, 0.06787744, 0.06689347, 0.01857092, 0.01451021,
       0.0096355 , 0.02290521, 0.02232303, 0.01227125, 0.02371998,
       0.00577067, 0.03045495, 0.01968819, 0.02447706, 0.00928838,
       0.01374109, 0.00211437, 0.01620473, 0.04439811, 0.03892214,
       0.01173182])

In [64]:
import lightgbm as lgb
train_data=lgb.Dataset(x_train,label=y_train)
#define parameters
params = {'learning_rate':0.1}
model= lgb.train(params, train_data, 100) 
y_pred=model.predict(x_test)
for i in range(len(x_test)):
    if y_pred[i]>=0.5: 
        y_pred[i]=1
    else: 
        y_pred[i]=0

print confusion_matrix(y_test, y_pred)

[[ 5  7  0]
 [ 0 19  0]
 [ 0  8  0]]


In [70]:
df.Corona_result.value_counts()

1    69
0    33
2    25
Name: Corona_result, dtype: int64

In [71]:
from sklearn.model_selection import GridSearchCV

In [81]:
parameters = {
              'C': [0.001,0.01,0.1,1,10,100,1000],
              'solver':['newton-cg', 'lbfgs', 'sag', 'saga' ], 
              'tol': [0.0001, 0.001], 
              'n_jobs': [-1]
             }
clf = GridSearchCV(LogisticRegression(penalty='l2', random_state=0), parameters)
clf.fit(x_train, y_train)
print sorted(clf.cv_results_.keys())

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'mean_train_score', 'param_C', 'param_n_jobs', 'param_solver', 'param_tol', 'params', 'rank_test_score', 'split0_test_score', 'split0_train_score', 'split1_test_score', 'split1_train_score', 'split2_test_score', 'split2_train_score', 'std_fit_time', 'std_score_time', 'std_test_score', 'std_train_score']


In [82]:
clf.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=0, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
def k_folds(model, n_folds, x, y, test):
    folds = StratifiedKFold(n_splits=5, shuffle=True, )